<a href="https://colab.research.google.com/github/mengchi501/Bag-of-Words/blob/data_preprocess/Bag_of_Words_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_dir = '/content/drive/MyDrive/Bag of Words Data'

os.listdir(data_dir)

['labeledTrainData.tsv',
 'testData.tsv',
 'unlabeledTrainData.tsv',
 'sampleSubmission.csv']

# Preprocess Data

전처리 작업

<s>1. 문장에 끼어있는 html tag들 제거</s>

<s>2. 단어 단위로 parse</s>

<s>3. 필요 없는 단어들 제거하기 위해 Stopword 사용</s>

4. vectorize

\+ <s>숫자/기호 제거</s>

\+ <s>표제어 추출(Lemmatization)</s>

In [7]:
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
labeled_train_df = pd.read_csv(os.path.join(data_dir, 'labeledTrainData.tsv'), sep='\t')
train_review = labeled_train_df.loc[:,'review']
train_sentiment = labeled_train_df.loc[:,'sentiment']

In [32]:
train_review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Beautiful Soup를 사용해서 train_review data에 섞여있는 html tag들을 제거해주자

Word2Vec Model을 import해서 token들을 vectorize하자

In [33]:
from bs4 import BeautifulSoup # html tag를 제거하는 기능이 포함되어 있는 라이브러리
import re # 정규 표현식을 활용할 수 있는 라이브러리
import nltk # Natural Language Tool Kit, 자연어 처리 라이브러리

nltk.download('stopwords') # 불용어 다운로드
nltk.download('wordnet') # wordnet 다운로드(Lemmatizer 포함)
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
sw = set(stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer() # Lemmatizer 생성

def review_to_words(review):
    only_text = BeautifulSoup(review).get_text() # html tag를 모두 제거
    only_alpha = re.sub("[^a-zA-Z]", " ", only_text) # 특수문자들 모두 제거
    words = only_alpha.lower().split() # 소문자로 변환하고 Tokenize
    tokens = [w for w in words if w not in sw] # 불용어 제거
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # 표제어 추출(Lemmatize)
    return tokens

In [35]:
train_review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [36]:
review_to_words(train_review[0][:10])
print(type(train_review))

<class 'pandas.core.series.Series'>


In [37]:
clean_train_review = train_review.apply(lambda review: review_to_words(review))

<ipython-input-34-b060bd73f213>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  only_text = BeautifulSoup(review).get_text() # html tag를 모두 제거


In [ ]:
from gensim.models import Word2Vec